# Imports

In [ ]:
#Basic operations
from __future__ import division
import time
import sys
import datetime
import gc
# To be able to import from lib directory
sys.path.append(r'..\lib\\')

#do not show warnings
import warnings
warnings.filterwarnings("ignore")

# Basic data analysis
import pandas as pd
import numpy as np
import seaborn as sns



import matplotlib.pyplot as plt
#import plotly for visualization
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go



# Import machine learning related libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report,confusion_matrix, SCORERS, f1_score, fbeta_score, precision_score, recall_score, roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC as svc
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

# Encoders
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.one_hot import OneHotEncoder 
from category_encoders.cat_boost import CatBoostEncoder

# Smoteen
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE, ADASYN

# LOCAL LIBRARIES
import SQL_connection as Jsc
import data_treatment as Adt

user='ALEJANDRO.LOZADA@RAPPI.COM'
password='zD13LqOhZDcPkEpgXacv8pA1'
# Defines the sql connection
con = Jsc.snowflake_connect(user=user,password=password)
# Initializes pyoff on notebook mode
pyoff.init_notebook_mode()


In [ ]:
query_launch = ''' 
SELECT T0.*
FROM [Jcountry]_writable.[Jtable] T0
WHERE DATE_FEATURE_PRED::DATE = '[Jdate]'::DATE
'''

In [ ]:
def create_update_predict(query_launch_=query_launch,
                          features_table_name='hvu_daily_feat',
                          predict_table_name= 'a',
                          countries={'CO':5, 'PE':5}):
    start_time = time.time()
    today = datetime.date.today()
    yesterday = today + datetime.timedelta(days=-1)

    for country in zip(countries.keys(), countries.values()):
        query_launch = query_launch_.replace('[Jtable]', features_table_name)
        query_launch = query_launch.replace('[Jdate]', str(yesterday))
        query_launch = query_launch.replace('[Jcountry]', country[0])

        try:
            alc_model = Jsc.pandas_df_from_snowflake_query(con, query_launch)
            print('Try')
        except ConnectionTimeOut as error:
            alc_model = None
            print('falló')
            pass

        if alc_model is not None:
            print('is not none ' + country[0])
            alc_model.columns = [i.upper() for i in alc_model.columns]
            numeric_feat = [num for num in alc_model.columns if 'ZZZ' in num]
            cate_feat_encode = [cat for cat in alc_model.columns if 'KKK' in cat]
            extra_feat = [extra for extra in alc_model.columns if 'EXX' in extra]

            alc_treat = alc_model[numeric_feat + cate_feat_encode + extra_feat]

            alc_treat[numeric_feat] = alc_treat[numeric_feat].astype(float)

            # Treat numeric null values
            #alc_treat = adt.nan_numeric(alc_treat, numeric_feat)
            #alc_treat = adt.nan_cate(alc_treat, cate_feat_encode)
            #alc_treat = adt.nan_cate(alc_treat, extra_feat, replacement='-1')

        else:
            print('is none')
            continue
    return alc_model

In [ ]:
alc_model = create_update_predict()

In [ ]:
alc_model

In [ ]:
if alc_model['LOG_VERIFICATION_CODE_RATIO_ZZZ'].isnull().all():
    print('wtf')

In [ ]:
~alc_model['LOG_VERIFICATION_CODE_RATIO_ZZZ'].isnull().all()

# Data

## Data generation

In [ ]:
#Functions
def delete(con,country,name):
    quer = 'DROP TABLE [Jcountry]_WRITABLE.' + str(name)
    quer = quer.replace('[Jcountry]',country)
    Jsc.execute_snowflake_query(con, quer, with_cursor=False)
    
def excecutef(quer,country,command,con):
    quer = quer.replace('[Jstatement]',command)
    quer = quer.replace('[Jcountry]',country)
    Jsc.execute_snowflake_query(con, quer, with_cursor=False)
    
def generate_data(query, 
                  conection, 
                  reset=False, 
                  countries = ['CO','BR','MX','CL','AR','PE','UY'], 
                  time_windows = ['1','3']):
    start_time = time.time()
    #statements
    if reset:
        con = conection
        for Jcountry in countries:
            print("--- Se originó centinela para " + Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            delete(con,Jcountry,'HIGH_VALUE_DATA')
            print("--- Se borró la tabla para " + Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            for Jtarget in time_windows:
                queryc = query
                queryc = queryc.replace('[Jvertical]',Jtarget)
                print("--- Se cargó el query para " + Jtarget + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))

                try:
                    excecutef(queryc,Jcountry,insert,con)
                    print("--- Se insertaron filas para " + Jvertical + " ---")
                    print("--- %s seconds ---\n" % round((time.time() - start_time),2))

                except:               
                    excecutef(queryc,Jcountry,create,con)
                    print("--- Se originó la tabla con " + Jvertical + " --- \nSu nombre es "+str(Jcountry)+"_WRITABLE.VERTICAL_CROSS_DATA_BACKUP")
                    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
    else:
        insert = 'INSERT INTO [Jcountry]_WRITABLE.HIGH_VALUE_DATA'
        create = 'CREATE TABLE [Jcountry]_WRITABLE.HIGH_VALUE_DATA AS'
        con = conection
        for Jcountry in countries:
            print("--- Se originó centinela para " + Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            for Jtarget in time_windows:
                queryc = query
                queryc = queryc.replace('[Jtarget]',Jtarget)
                print("--- Se cargó el query para " + Jtarget + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                try:
                    excecutef(queryc,Jcountry,insert,con)
                    print("--- Se insertaron filas para " + Jtarget + " ---")
                    print("--- %s seconds ---\n" % round((time.time() - start_time),2))

                except:   
                    print('Ya voy a crear')
                    excecutef(queryc,Jcountry,create,con)
                    print("--- Se originó la tabla con " + Jtarget + ". n\Su nombre es"+str(Jcountry)+"_WRITABLE.VERTICAL_CROSS_DATA_BACKUP ---")
                    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                    
def save_reporting(Jcountry, 
                   y_true, 
                   x_test , 
                   model, 
                   th, 
                   today, 
                   model_name, 
                   iterable, 
                   path='../data_1/', 
                   matrix_labels=[0,1]):
    
    from sklearn.metrics import roc_curve, auc, balanced_accuracy_score
    import six
    from matplotlib.backends.backend_pdf import PdfPages
    
    bbox=[0, 0, 1, 1]
    
    y_pre = model.predict_proba(x_test)[:,1]
    
    p = y_pre.copy()
    p[p<th] = 0
    p[p>=th] = 1
    
    #Balanced Accuracy
    Adt.save(Jcountry, balanced_accuracy_score(y_true, p), path , model_name+'/'+iterable+'/Reporting','balanced_acc'+'_'+today)
    
    fullReport = PdfPages(path+Jcountry+'/'+model_name+'/'+iterable+'/Reporting/full_report'+'_'+today+".pdf")


    #Adt.save(Jcountry, classification_report(p,y_true), path , model_name+'/'+iterable+'/Reporting','class_report'+'_'+today)

    #Whole report - I
    fig = plt.figure(figsize=(12,12))
    fig.suptitle('Reporte de resultados para EXP - 2019-12-20', fontsize=16, y=1.02)
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,1,2)
    ax3 = plt.subplot(2,2,2)
    
    
    #ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pre)
    roc_auc = auc(fpr, tpr)
    ax1.plot(fpr, tpr, lw=1, alpha=0.3,label='(AUC = %0.2f)'%roc_auc)
    ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    ax1.set_xlim([-0.05, 1.05])
    ax1.set_ylim([-0.05, 1.05])
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC curve ')
    ax1.legend(loc='best')
    
   
    
    #Lift Chart
    df_dict = {'actual': list (y_true), 'pred': list(y_pre)}
    df = pd.DataFrame(df_dict)
    pred_ranks = pd.qcut(df['pred'].rank(method='first'), 100, labels=False)
    actual_ranks = pd.qcut(df['actual'].rank(method='first'), 100, labels=False)
    pred_percentiles = df.groupby(pred_ranks).mean()
    actual_percentiles = df.groupby(actual_ranks).mean()
    
    ax2.set_title('Lift Chart',y=1)
    ax2.plot(np.arange(.01, 1.01, .01), np.array(pred_percentiles['pred']),
             color='darkorange', lw=2, label='Prediction')
    ax2.plot(np.arange(.01, 1.01, .01), np.array(pred_percentiles['actual']),
             color='navy', lw=2, linestyle='--', label='Actual')
    ax2.set_ylabel('Target Average')
    ax2.set_xlabel('Population Percentile')
    ax2.set_xlim([0.0, 1])
    ax2.set_ylim([0,0.05+max([max(np.array(pred_percentiles['pred'])),max(np.array(pred_percentiles['actual']))])])
    ax2.legend(loc="best")
    
    
    #Confusion Matrix
    array = confusion_matrix(y_true, p, matrix_labels)
    df_cm = pd.DataFrame(array, index = [i for i in matrix_labels],
                                columns = [i for i in matrix_labels])
    sns.heatmap(df_cm, annot=True, cmap='Blues',ax=ax3)
    ax3.set_title('Confusion Matrix ',y=1.08)
    #ax3.set_xticklabels([''] + matrix_labels)
    #ax3.set_yticklabels([''] + matrix_labels)
    ax3.set_xlabel('Predicted')
    ax3.set_ylabel('True')
    
    fig.tight_layout()
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
    fullReport.savefig()
    plt.close()
    
    

    #Whole report II
    fig1 = plt.figure(figsize=(12,12))
    fig1.suptitle('Reporte de resultados para EXP - 2019-12-20', fontsize=16, y=1.02)
    ax4 = plt.subplot(2,2,1)
    ax5 = plt.subplot(2,2,2)
    ax6 = plt.subplot(2,1,2)
    
    ax4.axis('off')
    ax5.axis('off')
    ax6.axis('off')
    
    #Important features
    #All
    arg_s = (-model.feature_importances_).argsort()
    data = pd.DataFrame(zip(x_test.columns[arg_s] ,model.feature_importances_[arg_s]),columns = ['COLUMNA','IMPORTANCIA'])
    data.IMPORTANCIA = round(data.IMPORTANCIA,3)
    data.COLUMNA = data['COLUMNA'].map(lambda x: x.replace('KKK','').replace('ZZZ','').replace('EXX','')) 
    data.COLUMNA = data['COLUMNA'].map(lambda x: '\n '.join(x[i:i + 15] for i in range(0, len(x), 15))) 
    
    #Best 10
    ax4.set_title('Best Features',y=1.02)
    best = ax4.table(cellText=data.head(10).values, bbox=bbox, colLabels=data.columns)
    best.auto_set_font_size(False)
    best.set_fontsize(14)
    
    #Worst 10
    ax5.set_title('Worst Features',y=1.02)
    worst = ax5.table(cellText=data.tail(10).values, bbox=bbox, colLabels=data.columns)
    worst.auto_set_font_size(False)
    worst.set_fontsize(14)
    
    
    #Classification report
    report = classification_report(y_true, p, output_dict=True)
    clas = pd.DataFrame(report).transpose()
    for i in clas.columns:
        clas[i] = round(clas[i],3)
        
    ax6.set_title('Classification Report',y=1.02)
    clasi = ax6.table(cellText=clas.values, bbox=bbox, rowLabels=clas.index, colLabels=clas.columns)
    
    clasi.auto_set_font_size(False)
    clasi.set_fontsize(14)

    #Tables pretty
    for mpl_table in [best,worst,clasi]:
        for k, cell in six.iteritems(mpl_table._cells):
            cell.set_edgecolor('w')
            if k[0] == 0 or k[1] < 0:
                cell.set_text_props(weight='bold', color='w')
                cell.set_facecolor('#40466e')
            else:
                cell.set_facecolor(['#f1f1f2', 'w'][k[0]%len(['#f1f1f2', 'w']) ])

    fig1.tight_layout()
    
    
    fig1.subplots_adjust(left=0.2, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
    fullReport.savefig()
    plt.close()
    fullReport.close()
            

In [ ]:
#query_microzones
query_microzones = '''
[Jstatement]

WITH

PARAMS AS (

    SELECT 14 AS HISTORY_MONTHS,
           8 AS TARGET_WINDOW,
           2 AS BACK_WINDOW,
           16 AS TOP_DEFINITION,
           'BR' AS COUNTRY,
           17 AS TASA_MX,
           2800 AS TASA_CO,
           46 AS TASA_AR,
           31 AS TASA_UY,
           3.2 AS TASA_PE,
           620 AS TASA_CL,
           3.2 AS TASA_BR,
           0.1 AS AVG_TAKE_RATE
    )

,COMMISSIONS AS (

        SELECT COUNTRY
              ,ORDER_ID
              ,MARKUP_USD
              ,SERVICE_FEE_USD
              ,VALUE_COMMISSION_STORE_USD
              ,VALUE_COMMISSION_RESTAURANT_USD
              ,RAPPICASH_SURCHARGE_USD
              ,OTHER_SURCHARGES_USD
              ,MARKUP_USD +
                SERVICE_FEE_USD +
                VALUE_COMMISSION_STORE_USD +
                VALUE_COMMISSION_RESTAURANT_USD +
                RAPPICASH_SURCHARGE_USD +
                OTHER_SURCHARGES_USD AS COMMISSIONS
        FROM GLOBAL_FINANCES.GLOBAL_ORDER_DETAILS
        WHERE COUNTRY ILIKE (SELECT COUNTRY FROM PARAMS)

    )

,DISCOUNTS  AS (

    SELECT SOURCE_TABLE AS COUNTRY
          ,ORDER_ID
          , zeroifnull(PROMOT_DISC_RETENTION) AS PROMOT_DISC_RETENTION
          , zeroifnull(PROMOT_DISCOUNTS_REACTIVATION) AS PROMOT_DISCOUNTS_REACTIVATION
          , zeroifnull(PROMOT_DISCOUNTS_COMPENSA) AS PROMOT_DISCOUNTS_COMPENSA
          , zeroifnull(PROMOT_DISC_RETENTION) +
            zeroifnull(PROMOT_DISCOUNTS_REACTIVATION) +
            zeroifnull(PROMOT_DISCOUNTS_COMPENSA) AS DISCOUNTS_TO_REVENUE
    FROM BR_WRITABLE.TBL_STG_FACT_UE_MKT A
    LEFT JOIN GLOBAL_FINANCES.TBL_DIM_GEOGRAPHY_T1 B ON (A.SK_GEOGRAPHY = B.SK_GEOGRAPHY)
    WHERE SOURCE_TABLE  ILIKE (SELECT COUNTRY FROM PARAMS)

     )

,SEMIFINAL AS ( 
    SELECT '[Jdate]'::DATE AS DATE
           ,MICROZONE_ID
           --REVENUE--
           ,AVG(REVENUE) AS AVG_REV_ORD
           ,MEDIAN(REVENUE) AS MED_REV_ORD
           ,STDDEV(REVENUE) AS STD_REV_ORD
           ,SUM(REVENUE) AS TOT_REV
           --ORDERS MICRO--
           ,COUNT(DISTINCT ORDER_ID) AS TOT_ORD
           --USERS MICRO--
           ,COUNT(DISTINCT APPLICATION_USER_ID) AS TOT_USERS       
           --USERS-ORD-REV MICRO--
           ,ROUND(TOT_ORD/TOT_USERS,4) AS AVG_ORD_USER
           ,ROUND(TOT_REV/TOT_USERS,4) AS AVG_REV_USER

    FROM( 
        SELECT  O.APPLICATION_USER_ID
               ,O.COUNTRY
               ,O.ORDER_ID
               ,COUNT(DISTINCT O.ORDER_ID) OVER (PARTITION BY O.APPLICATION_USER_ID) AS NUM_ORD
               ,O.MICROZONE_ID
               ,O.CREATED_AT
               ,ZEROIFNULL(MARKUP_USD) AS MARKUP_USD
               ,ZEROIFNULL(SERVICE_FEE_USD) AS SERVICE_FEE_USD
               ,ZEROIFNULL(VALUE_COMMISSION_STORE_USD) AS VALUE_COMMISSION_STORE_USD
               ,ZEROIFNULL(VALUE_COMMISSION_RESTAURANT_USD) AS VALUE_COMMISSION_RESTAURANT_USD
               ,ZEROIFNULL(RAPPICASH_SURCHARGE_USD) AS RAPPICASH_SURCHARGE_USD
               ,ZEROIFNULL(OTHER_SURCHARGES_USD) AS OTHER_SURCHARGES_USD
               ,ZEROIFNULL(COMMISSIONS) AS COMMISSION
               ,ZEROIFNULL(DISCOUNTS_TO_REVENUE) AS DISCOUNTS_TO_REVENUE
               ,ZEROIFNULL(COMMISSIONS) + ZEROIFNULL(DISCOUNTS_TO_REVENUE) AS REVENUE
        FROM GLOBAL_FINANCES.GLOBAL_ORDERS AS O
        LEFT JOIN COMMISSIONS AS C ON (O.COUNTRY = C.COUNTRY AND O.ORDER_ID = C.ORDER_ID)
        LEFT JOIN DISCOUNTS AS D ON (O.COUNTRY = D.COUNTRY AND O.ORDER_ID = D.ORDER_ID)
        WHERE O.STATE_TYPE = 'FINISHED'
        AND COALESCE(O.STORE_TYPE, 'WHIMP') <> 'grin'
        AND NOT IS_SUBSCRIPTION
        AND GMV_USD > 0
        AND O.COUNTRY ILIKE (SELECT COUNTRY FROM PARAMS)
        AND O.CREATED_AT::DATE <= '[Jdate]'::DATE
        AND O.CREATED_AT::DATE >= DATEADD('MONTH', -(SELECT BACK_WINDOW FROM PARAMS), '[Jdate]'::DATE)::DATE)
        GROUP BY 1, 2 )
        
SELECT T0.*
      ,CASE WHEN PERC_MICRO_REV IN (5) AND PERC_MICRO_ORD IN (4,5) THEN 1 ELSE 0 END AS HV_MICROZONE
FROM ( 
SELECT T0.*
      ,NTILE(5) OVER (PARTITION BY NULL ORDER BY AVG_REV_USER) AS PERC_MICRO_REV
      ,NTILE(5) OVER (PARTITION BY NULL ORDER BY AVG_ORD_USER) AS PERC_MICRO_ORD
FROM SEMIFINAL T0 ) T0






'''

In [ ]:
#Microzone data
insert = 'INSERT INTO BR_WRITABLE.high_value_user_revenue_data_microzone_final'
create = 'CREATE TABLE BR_WRITABLE.high_value_user_revenue_data_microzone_final AS'

date_stop = datetime.date(2019,6,29)
date_start =datetime.date(2019,6,29) #2019-01-01
missing_days = date_stop - date_start
missing_days = 1#missing_days.days


for i in range(1, missing_days + 1):
    query = query_microzones
    date = date_start + datetime.timedelta(days=i)
    query = query.replace('[Jdate]', str(date))
    try:
        queryi = query.replace('[Jstatement]',insert)
        Jsc.execute_snowflake_query(con, queryi)
        print('-- Se intentará insertar por primera vez en: ' +str(date)+ ' --')
    except:
        queryc = query.replace('[Jstatement]',create)
        Jsc.execute_snowflake_query(con, queryc)
        print('-- Se creó por primera vez en: '+str(date)+ ' --')
        pass
    print('-- Insertado con éxito para: ' + str(date) + ' --')

In [ ]:
#qh
query_high = '''

SELECT T0.*,
       LEFT(RIGHT(try_to_number(APPLICATION_USER_ID),UNIFORM(2,3,HOUR(CURRENT_TIMESTAMP))),2) + LEFT(RIGHT(UNIFORM(1.00::FLOAT,100::FLOAT,HOUR(CURRENT_TIMESTAMP)),5),2) AS RANDOM
FROM [Jcountry]_WRITABLE.[Jtable] T0
WHERE [Jiterable_name] = 6--[Jiterable]
ORDER BY RANDOM
limit 200000


'''

In [ ]:
#query_test
query_test = '''

SELECT T0.*
FROM [Jcountry]_WRITABLE.HIGH_VALUE_USER_DATA_BACKTEST T0
WHERE TIME_WINDOW = [Jtime]

'''

In [ ]:
#query_th
query_th = '''

SELECT *
FROM BR_WRITABLE.high_value_user_revenue_data

'''


In [ ]:
ALC_th = Jsc.pandas_df_from_snowflake_query(con, query_th)

In [ ]:
ALC_treat = ALC_th.copy()

In [ ]:
#EXCLUDE
EXCLUDE = ['QTY_SELECT_LOGIN_TOTAL_ZZZ',
           'QTY_SELECT_LOGIN_ZZZ',
           'ORDER_PLACED_CONFIRMED_ERROR_FOUND_ZZZ',
           'QTY_ORDER_PC_ERROR_TOTAL_ZZZ',
           'QTY_COUPON_ERROR_TOTAL_ZZZ',
           'QTY_COUPON_ERROR_ZZZ',
           'COMPLETE_SIGN_ERROR_FOUND_ZZZ',
           'QTY_COMPLETE_SIGN_ERROR_TOTAL_ZZZ',
           'COMPLETE_REGISTRATION_FOUND_ZZZ',
           'SESSIONS_FOUND_EXX',
           'QTY_COMPLETE_LOGIN_ERROR_ZZZ',
           'COMPLETE_LOGIN_FOUND_ZZZ',
           'QTY_COMPLETE_LOGIN_ZZZ',
           'ADDRESS_CHECK_ERROR_FOUND_ZZZ',
           'QTY_ADDRESS_CHECK_ERROR_ZZZ',
           'ADDRESS_CHANGED_FOUND_ZZZ',
           'QTY_ADDRESS_CHANGED_ZZZ',
           'ADD_ADDRESS_ERROR_FOUND_ZZZ',
           'QTY_VIEW_HOME_ERROR_ZZZ',
           'COUPON_ERROR_FOUND_ZZZ',
           'QTY_COMPLETE_REGISTRATION_ZZZ',
           'COMPLETE_REGISTER_ERROR_FOUND_ZZZ',
           'QTY_COMP_REGI_ERROR_TOTAL_ZZZ',
           'QTY_COMPLETE_REGISTER_ERROR_ZZZ',
           'COMPLETE_LOGIN_ERROR_FOUND_ZZZ',
           'QTY_ADD_ADDRESS_ERROR_TOTAL_ZZZ',
           'MODE_OS_KKK',
           'QTY_ADD_ADDRESS_ERROR_ZZZ',
           'VERIFICATION_CODE_ERROR_FOUND_ZZZ',
           'QTY_VERIFICATION_CODE_SUCCESS_ZZZ',
           'CARD_BRAND_AGG_KKK', 'ACTUAL_TIP_SHARE_ZZZ',
           'DIF_ORGANIC_SHARE_ZZZ', 'MODE_PAYMENT_METHOD_KKK', 'TRIES_ZZZ',
           'DIF_FIRST_DEVICE_YEAR_ZZZ', 'REGISTER_TRIES_ZZZ',
           'NUMBER_OF_CC_ZZZ', 'LOG_AVG_DURATION_MS_AVG_TOTAL_ZZZ',
           'WEEK_REGISTER_CC_EXX', 'VIEW_HOME_ERROR_FOUND_ZZZ',
           'QTY_VIEW_HOME_ERROR_TOTAL_ZZZ',
           'VERIFICATION_CODE_SUCCESS_FOUND_ZZZ',
           'LOG_VERIFICATION_CODE_RATIO_ZZZ', 'VERIFICATION_CODE_RATIO_ZZZ',
           'QTY_VERIFICATION_CODE_ERROR_ZZZ', 'QTY_COMPLETE_SIGN_ERROR_ZZZ',
           'RELATIVE_DIFF_MS_DURATION_ZZZ', 'MAX_TIP_SHARE_ZZZ',
           'SAME_COUNTRY_EXX',
           'HIGH_VALUE_USER_EXX']

NEW_OUT = ['AVG_REV_ORD_MIC_ZZZ',
           'MED_REV_ORD_MIC_ZZZ',
           'TOT_REV_MIC_ZZZ',
           'TOT_ORD_MIC_ZZZ',
           'TOT_USERS_MIC_ZZZ',
           'PERC_MICRO_REV_MIC_EXX',
           'PERC_MICRO_ORD_MIC_EXX',
           'HV_MICROZONE_MIC_EXX',
           'AVG_ORD_USER_MIC_ZZZ',  # Micro
           'AVG_REV_USER_MIC_ZZZ',  # Micro
           'STD_REV_ORD_MIC_ZZZ']  # Micro

EXCLUDE = EXCLUDE + NEW_OUT

In [ ]:
numeric_feat = [num for num in ALC_treat.columns if 'ZZZ' in num]
cate_feat_encode = [cat for cat in ALC_treat.columns if 'KKK' in cat]
extra_feat = [extra for extra in ALC_treat.columns if 'EXX' in extra]
        
numeric_feat = [num for num in numeric_feat if num not in EXCLUDE]
cate_feat_encode = [cat for cat in cate_feat_encode if cat not in EXCLUDE]
extra_feat = [extra for extra in extra_feat if extra not in EXCLUDE]

In [ ]:
ALC_treat = Adt.Nan_Numeric(ALC_treat,numeric_feat)
ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)
ALC_treat = Adt.Nan_Cate(ALC_treat,extra_feat,replacement=-1)

X = ALC_treat[numeric_feat+extra_feat+cate_feat_encode]



In [ ]:
country='BR'
time='5'
path_cate_high = '../data_1/{}/high_value_users_new/{}/cate_high_2020-04-23'.format(country,str(time))
path_cate_low = '../data_1/{}/high_value_users_new/{}/cate_low_2020-04-23'.format(country,str(time))
path_sc = '../data_1/{}/high_value_users_new/{}/scaler_2020-04-23'.format(country,str(time))
path_mod = '../data_1/{}/high_value_users_new/{}/mod_2020-04-23'.format(country,str(time))
path_th = '../data_1/{}/high_value_users_new/{}/th_2020-04-23'.format(country,str(time))
        
        
        
        
cate_high = Adt.read(path_cate_high)
cate_low = Adt.read(path_cate_low)
sc = Adt.read(path_sc)
mod = Adt.read(path_mod)
th = Adt.read(path_th)

In [ ]:
X[numeric_feat] = pd.DataFrame(sc.transform(X[numeric_feat]), columns=numeric_feat, index = X.index)
X = cate_high.transform(X)
X = cate_low.transform(X)



In [ ]:
y_pred = mod.predict_proba(X)[:,1]

In [ ]:
ALC_treat['Y_PRED'] =y_pred

In [ ]:
ALC_treat.head()

In [ ]:
ALC_treat['Y_PRED_rank'] = pd.qcut(ALC_treat['Y_PRED'].rank(method='first'), 100, labels=False)

In [ ]:
th_50 = min(ALC_treat[ALC_treat['Y_PRED_rank']>=50].Y_PRED)

In [ ]:
ALC_treat[ALC_treat['Y_PRED_rank']>=50].shape[0]/ALC_treat.shape[0]

In [ ]:
th_50

In [ ]:
y_predicted = ALC_treat['Y_PRED']
y_predicted[y_predicted<th_50] = 0
y_predicted[y_predicted>=th_50] = 1

In [ ]:
sns.countplot(y_predicted)

In [ ]:
sns.distplot(ALC_treat['Y_PRED'])

In [ ]:
ALC_treat['MONTH'] = [i.month for i in ALC_treat.FIRST_ACTUAL_ORDER]

In [ ]:
sns.distplot(ALC_treat['MONTH'])

In [ ]:
ALC_treat.head()

In [ ]:
all_feat = numeric_feat+extra_feat+cate_feat_encode

for i in all_feat:
    sns.distplot(ALC_treat[i])
    plt.show()
    plt.close()

In [ ]:
ALC_treat[ALC_treat.APPLICATION_USER_ID==39344]

In [ ]:
print(classification_report(ALC_treat.HIGH_VALUE_USER_EXX,y_predicted ))

In [ ]:
for country in countries:
    
    query1 = query_test.replace('[Jcountry]',country)
    
    query2 = query1.replace('[Jtime]',str(1))
    ALC_test_ = Jsc.pandas_df_from_snowflake_query(con, query2)
    print('Cargados los datos crudos para ' + country)
    
    numeric_feat = [num for num in ALC_test_.columns if 'ZZZ' in num]
    cate_feat_encode = [cat for cat in ALC_test_.columns if 'KKK' in cat]
    extra_feat = [extra for extra in ALC_test_.columns if 'EXX' in extra]
        
    numeric_feat = [num for num in numeric_feat if num not in extra]
    

    ALC_treat_ = ALC_test_[numeric_feat + cate_feat_encode + extra_feat]

    ALC_treat_[numeric_feat] = ALC_treat_[numeric_feat].astype(float)

    #Treat numeric null values
    ALC_treat_ = Adt.Nan_Numeric(ALC_treat_,numeric_feat)
    ALC_treat_ = Adt.Nan_Cate(ALC_treat_,cate_feat_encode)
    ALC_treat_ = Adt.Nan_Cate(ALC_treat_,extra_feat,replacement=-1)
        
    X_ = ALC_treat_.drop("HIGH_VALUE_USER_EXX",axis=1)
    
    
    for time in time_windows:
        
        query = query_test.replace('[Jcountry]',country)
        query = query.replace('[Jtime]',str(time))
        
        print(query)

        ALC_test = Jsc.pandas_df_from_snowflake_query(con, query)
        
        print('Cargados los datos: ' +str(time))
        
        path_cate_high = '../data_1/{}/high_value_users/{}/cate_high_2020-03-27'.format(country,str(time))
        path_cate_low = '../data_1/{}/high_value_users/{}/cate_low_2020-03-27'.format(country,str(time))
        path_sc = '../data_1/{}/high_value_users/{}/scaler_2020-03-27'.format(country,str(time))
        path_mod = '../data_1/{}/high_value_users/{}/mod_2020-03-27'.format(country,str(time))
        path_th = '../data_1/{}/high_value_users/{}/th_2020-03-27'.format(country,str(time))
        
        
        
        
        cate_high = Adt.read(path_cate_high)
        cate_low = Adt.read(path_cate_low)
        sc = Adt.read(path_sc)
        mod = Adt.read(path_mod)
        th = Adt.read(path_th)
        
        print('Cargados los pkl')
        

        X = X_.copy()
        X[numeric_feat] = pd.DataFrame(sc.transform(X[numeric_feat]), columns=numeric_feat, index = X.index)
        X = cate_high.transform(X)
        X = cate_low.transform(X)
        
        y = ALC_test["HIGH_VALUE_USER_EXX"]
        
        print('Transformaciones listas')
        
        
        save_reporting(country, 
                        y, 
                        X , 
                        mod, 
                        th, 
                        today='back_test_cry_4', 
                        model_name='high_value_users', 
                        iterable=str(time), 
                        path='../data_1/', 
                        matrix_labels=[0,1])
        
        
        print('Guardado el reporting')
        
        ALC_test_['PRED_' + str(time)] = mod.predict_proba(X)[:, 1]

        ALC_test_['TRH_' + str(time)] = [th] * ALC_test.shape[0]
        
        ALC_test_['REAL_HVU_' + str(time)] = ALC_test["HIGH_VALUE_USER_EXX"]
        
        print(ALC_test.columns)
        
    export = [i for i in ALC_test_.columns if 'PRED_' in i] + \
             [i for i in ALC_test_.columns if 'TRH_' in i] + \
             [i for i in ALC_test_.columns if 'REAL_HVU_' in i] + \
             ['APPLICATION_USER_ID'] + ['FIRST_ACTUAL_ORDER']
    
    '''
    resp = Jsc.write_snowflake_table(data=ALC_test_[export],
                                        table_name='high_value_user_pred',
                                        Jcountry = country,
                                        user=user,
                                        password = password,
                                        if_exists_then_wat='replace')
    '''                                            
    print('Exportado a HIGH_VALUE_USER_PRED para ' + country)


# Automation

In [ ]:
#More_Functions

def save_reporting(Jcountry, 
                   y_true, 
                   x_test , 
                   model, 
                   th, 
                   today, 
                   model_name, 
                   iterable, 
                   path='../data_1/', 
                   matrix_labels=[0,1]):
    
    from sklearn.metrics import roc_curve, auc, balanced_accuracy_score
    import six
    from matplotlib.backends.backend_pdf import PdfPages
    
    bbox=[0, 0, 1, 1]
    
    y_pre = model.predict_proba(x_test)[:,1]
    
    p = y_pre.copy()
    p[p<th] = 0
    p[p>=th] = 1
    
    #Balanced Accuracy
    Adt.save(Jcountry, balanced_accuracy_score(y_true, p), path , model_name+'/'+iterable+'/Reporting','balanced_acc'+'_'+today)
    
    fullReport = PdfPages(path+Jcountry+'/'+model_name+'/'+iterable+'/Reporting/full_report'+'_'+today+".pdf")


    #Adt.save(Jcountry, classification_report(p,y_true), path , model_name+'/'+iterable+'/Reporting','class_report'+'_'+today)

    #Whole report - I
    fig = plt.figure(figsize=(12,12))
    fig.suptitle('Reporte de resultados para EXP - 2019-12-20', fontsize=16, y=1.02)
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,1,2)
    ax3 = plt.subplot(2,2,2)
    
    
    #ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pre)
    roc_auc = auc(fpr, tpr)
    ax1.plot(fpr, tpr, lw=1, alpha=0.3,label='(AUC = %0.2f)'%roc_auc)
    ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    ax1.set_xlim([-0.05, 1.05])
    ax1.set_ylim([-0.05, 1.05])
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC curve ')
    ax1.legend(loc='best')
    
   
    
    #Lift Chart
    df_dict = {'actual': list (y_true), 'pred': list(y_pre)}
    df = pd.DataFrame(df_dict)
    pred_ranks = pd.qcut(df['pred'].rank(method='first'), 100, labels=False)
    actual_ranks = pd.qcut(df['actual'].rank(method='first'), 100, labels=False)
    pred_percentiles = df.groupby(pred_ranks).mean()
    actual_percentiles = df.groupby(actual_ranks).mean()
    
    ax2.set_title('Lift Chart',y=1)
    ax2.plot(np.arange(.01, 1.01, .01), np.array(pred_percentiles['pred']),
             color='darkorange', lw=2, label='Prediction')
    ax2.plot(np.arange(.01, 1.01, .01), np.array(pred_percentiles['actual']),
             color='navy', lw=2, linestyle='--', label='Actual')
    ax2.set_ylabel('Target Average')
    ax2.set_xlabel('Population Percentile')
    ax2.set_xlim([0.0, 1])
    ax2.set_ylim([0,0.05+max([max(np.array(pred_percentiles['pred'])),max(np.array(pred_percentiles['actual']))])])
    ax2.legend(loc="best")
    
    
    #Confusion Matrix
    array = confusion_matrix(y_true, p, matrix_labels)
    df_cm = pd.DataFrame(array, index = [i for i in matrix_labels],
                                columns = [i for i in matrix_labels])
    sns.heatmap(df_cm, annot=True, cmap='Blues',ax=ax3)
    ax3.set_title('Confusion Matrix ',y=1.08)
    #ax3.set_xticklabels([''] + matrix_labels)
    #ax3.set_yticklabels([''] + matrix_labels)
    ax3.set_xlabel('Predicted')
    ax3.set_ylabel('True')
    
    fig.tight_layout()
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
    fullReport.savefig()
    plt.close()
    
    

    #Whole report II
    fig1 = plt.figure(figsize=(12,12))
    fig1.suptitle('Reporte de resultados para EXP - 2019-12-20', fontsize=16, y=1.02)
    ax4 = plt.subplot(2,2,1)
    ax5 = plt.subplot(2,2,2)
    ax6 = plt.subplot(2,1,2)
    
    ax4.axis('off')
    ax5.axis('off')
    ax6.axis('off')
    
    #Important features
    #All
    arg_s = (-model.feature_importances_).argsort()
    data = pd.DataFrame(zip(x_test.columns[arg_s] ,model.feature_importances_[arg_s]),columns = ['COLUMNA','IMPORTANCIA'])
    data.IMPORTANCIA = round(data.IMPORTANCIA,3)
    data.COLUMNA = data['COLUMNA'].map(lambda x: x.replace('KKK','').replace('ZZZ','').replace('EXX','')) 
    data.COLUMNA = data['COLUMNA'].map(lambda x: '\n '.join(x[i:i + 15] for i in range(0, len(x), 15))) 
    
    #Best 10
    ax4.set_title('Best Features',y=1.02)
    best = ax4.table(cellText=data.head(10).values, bbox=bbox, colLabels=data.columns)
    best.auto_set_font_size(False)
    best.set_fontsize(14)
    
    #Worst 10
    ax5.set_title('Worst Features',y=1.02)
    worst = ax5.table(cellText=data.tail(10).values, bbox=bbox, colLabels=data.columns)
    worst.auto_set_font_size(False)
    worst.set_fontsize(14)
    
    
    #Classification report
    report = classification_report(y_true, p, output_dict=True)
    clas = pd.DataFrame(report).transpose()
    for i in clas.columns:
        clas[i] = round(clas[i],3)
        
    ax6.set_title('Classification Report',y=1.02)
    clasi = ax6.table(cellText=clas.values, bbox=bbox, rowLabels=clas.index, colLabels=clas.columns)
    
    clasi.auto_set_font_size(False)
    clasi.set_fontsize(14)

    #Tables pretty
    for mpl_table in [best,worst,clasi]:
        for k, cell in six.iteritems(mpl_table._cells):
            cell.set_edgecolor('w')
            if k[0] == 0 or k[1] < 0:
                cell.set_text_props(weight='bold', color='w')
                cell.set_facecolor('#40466e')
            else:
                cell.set_facecolor(['#f1f1f2', 'w'][k[0]%len(['#f1f1f2', 'w']) ])

    fig1.tight_layout()
    
    
    fig1.subplots_adjust(left=0.2, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
    fullReport.savefig()
    plt.close()
    fullReport.close()
    
def export_train(query_promo:str =None, 
                 model_name:str =None,
                 table_name_train:str =None,
                 iterable_name:str =None,
                 target_name:str =None,
                 resample:bool =False,
                 resampler:object =SMOTE(random_state=0),
                 metric:object =f1_score,
                 path:str ='../data_1/',
                 model:object =XGBClassifier(), 
                 countries:list =['UY','MX','BR','AR','CL','PE','CO']):

    start_time = time.time()
    today = time.strftime('%Y-%m-%d', time.localtime(start_time))
    print("--- Se hará el entrenamiento el día " + today + " ---")
    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
    
    #exclude1
    exclude1 =  ['QTY_SELECT_LOGIN_TOTAL_ZZZ',
                 'QTY_SELECT_LOGIN_ZZZ',
                 'ORDER_PLACED_CONFIRMED_ERROR_FOUND_ZZZ',
                 'QTY_ORDER_PC_ERROR_TOTAL_ZZZ',
                 'QTY_COUPON_ERROR_TOTAL_ZZZ',
                 'QTY_COUPON_ERROR_ZZZ',
                 'COMPLETE_SIGN_ERROR_FOUND_ZZZ',
                 'QTY_COMPLETE_SIGN_ERROR_TOTAL_ZZZ',
                 'COMPLETE_REGISTRATION_FOUND_ZZZ',
                 'SESSIONS_FOUND_EXX',
                 'QTY_COMPLETE_LOGIN_ERROR_ZZZ',
                 'COMPLETE_LOGIN_FOUND_ZZZ',
                 'QTY_COMPLETE_LOGIN_ZZZ',
                 'ADDRESS_CHECK_ERROR_FOUND_ZZZ',
                 'QTY_ADDRESS_CHECK_ERROR_ZZZ',
                 'ADDRESS_CHANGED_FOUND_ZZZ',
                 'QTY_ADDRESS_CHANGED_ZZZ',
                 'ADD_ADDRESS_ERROR_FOUND_ZZZ',
                 'QTY_VIEW_HOME_ERROR_ZZZ',
                 'COUPON_ERROR_FOUND_ZZZ',
                 'QTY_COMPLETE_REGISTRATION_ZZZ',
                 'COMPLETE_REGISTER_ERROR_FOUND_ZZZ',
                 'QTY_COMP_REGI_ERROR_TOTAL_ZZZ',
                 'QTY_COMPLETE_REGISTER_ERROR_ZZZ',
                 'COMPLETE_LOGIN_ERROR_FOUND_ZZZ',
                 'QTY_ADD_ADDRESS_ERROR_TOTAL_ZZZ',
                 'MODE_OS_KKK',
                 'QTY_ADD_ADDRESS_ERROR_ZZZ',
                 'VERIFICATION_CODE_ERROR_FOUND_ZZZ',
                 'QTY_VERIFICATION_CODE_SUCCESS_ZZZ',
                 'CARD_BRAND_AGG_KKK', 
                 'ACTUAL_TIP_SHARE_ZZZ',
                 'DIF_ORGANIC_SHARE_ZZZ', 
                 'MODE_PAYMENT_METHOD_KKK', 
                 'TRIES_ZZZ',
                 'DIF_FIRST_DEVICE_YEAR_ZZZ', 
                 'REGISTER_TRIES_ZZZ',
                 'NUMBER_OF_CC_ZZZ', 
                 'LOG_AVG_DURATION_MS_AVG_TOTAL_ZZZ',
                 'WEEK_REGISTER_CC_EXX', 
                 'VIEW_HOME_ERROR_FOUND_ZZZ',
                 'QTY_VIEW_HOME_ERROR_TOTAL_ZZZ',
                 'VERIFICATION_CODE_SUCCESS_FOUND_ZZZ',
                 'LOG_VERIFICATION_CODE_RATIO_ZZZ', 
                 'VERIFICATION_CODE_RATIO_ZZZ',
                 'QTY_VERIFICATION_CODE_ERROR_ZZZ', 
                 'QTY_COMPLETE_SIGN_ERROR_ZZZ',
                 'RELATIVE_DIFF_MS_DURATION_ZZZ', 
                 'MAX_TIP_SHARE_ZZZ',
                 'SAME_COUNTRY_EXX']

    for Jcountry in countries:
        query_promo = query_promo.replace('[Jcountry]',Jcountry)
        query_promo = query_promo.replace('[Jtable]', table_name_train)
        query_promo = query_promo.replace('[Jiterable_name]', iterable_name)
        print("--- Se ha cargado el query para " + Jcountry + " ---")
        print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
        query_iterable = '''

        SELECT DISTINCT
               [Jiterable_name]
        FROM [Jcountry]_WRITABLE.[Jtable] T0

        '''
        
        query_iterable = query_iterable.replace('[Jcountry]',Jcountry)
        query_iterable = query_iterable.replace('[Jtable]', table_name_train)
        query_iterable = query_iterable.replace('[Jiterable_name]', iterable_name)
        print("--- Se ha cargado el query para " + Jcountry + " ---")
        print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
        ALC_iterable = Jsc.pandas_df_from_snowflake_query(con, query_iterable)
        print("--- Se corrió el query para " + Jcountry + " ---")
        print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
        iterables = ALC_iterable[iterable_name].drop_duplicates().values
        print("--- Se hará procesamiento para las iterables " + iterables + " ---")
        print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        

        for iterable in iterables:
         
            query_promo_ite = query_promo.replace('[Jiterable]',str(iterable))

            ALC_model = Jsc.pandas_df_from_snowflake_query(con, query_promo_ite)
            print(ALC_model.shape)
            print("--- Se corrió el query para " + Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))

            numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]
            cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
            extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra]
            
            numeric_feat = [num for num in numeric_feat if num not in exclude1]
            cate_feat_encode = [cat for cat in cate_feat_encode if cat not in exclude1]
            extra_feat = [extra for extra in extra_feat if extra not in exclude1]

            ALC_treat = ALC_model[numeric_feat + cate_feat_encode + extra_feat]
            
            del ALC_model
            gc.collect()
            
            #Treat numeric null values
            ALC_treat = Adt.Nan_Numeric(ALC_treat,numeric_feat)
            print("--- Se llenaron vacíos numéricos con el promedio para " + str(iterable) +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)
            ALC_treat = Adt.Nan_Cate(ALC_treat,extra_feat,replacement=-1)
            print("--- Se llenaron vacíos categóricos con el 'Otro' para " + str(iterable) +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))

            #Data processing
            #Numeric scaling
            sc = StandardScaler()
            sc.fit(ALC_treat[numeric_feat])        
            ALC_treat[numeric_feat] = pd.DataFrame(sc.transform(ALC_treat[numeric_feat]), columns=numeric_feat, index = ALC_treat.index)
            print("--- Se hizo el escalamiento de variables numéricas para " + iterable +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
            #Define X and Target
            y = ALC_treat[target_name]
            X = ALC_treat.drop([target_name],axis=1)
            print("--- Se definió X y Y para " + iterable +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            
            del ALC_treat
            gc.collect()
            
            #Split training and test
            X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 27, stratify=y)
            print("--- Se separó entre train y test para " + iterable +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))   

            print(sum(Y_test)/len(Y_test))

            del X, y
            gc.collect()
            
            #Category encoding
            high_cardinality = []
            low_cardinality = []
            
            for cat_column in cate_feat_encode:
                cates = len(X_train[str(cat_column)].drop_duplicates().values)
                if (cates/len(cate_feat_encode)) > 100:
                    high_cardinality.append(cat_column)
                else:
                    low_cardinality.append(cat_column)
                    
            cate_low = CatBoostEncoder(cols = low_cardinality, drop_invariant=False)
            cate_low.fit(X_train, Y_train, cols = low_cardinality)
            X_train = cate_low.transform(X_train, Y_train)
                    
            cate_high = CatBoostEncoder(cols = high_cardinality, drop_invariant=False)
            cate_high.fit(X_train, Y_train, cols = high_cardinality)
            X_train = cate_high.transform(X_train, Y_train)
            
            
            X_test = cate_high.transform(X_test)
            X_test = cate_low.transform(X_test)
            #cate_low = OneHotEncoder(use_cat_names=True)
            #cate_low.fit(X,y,cols=cate_feat_encode)
            
            print("--- Se codificaron variables numéricas para " + iterable +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))


            #Save scaler for prediction
            Adt.save(Jcountry, sc, path, model_name+'/'+iterable, 'scaler_{}'.format(today))
            Adt.save(Jcountry, cate_high, path, model_name+'/'+iterable, 'cate_high_{}'.format(today))
            Adt.save(Jcountry, cate_low, path, model_name+'/'+iterable, 'cate_low_{}'.format(today))
            print("--- Se guardaron el escalador de numéricas y el codificador de categóricas " + iterable +' en '+ Jcountry + " ---")
            print("--- %s seconds ---\n" % round((time.time() - start_time),2))
            
            if X_train.shape[0]>5000:
                
                print(sum(Y_train)/len(Y_train))
            
                if resample:
                
                    #Resampling
                    sampler = resampler#SMOTE(random_state=0)
                    
                    X_train_r, Y_train_r = sampler.fit_resample(X_train, Y_train)
                    
                    X_train = pd.DataFrame(X_train_r, columns=X_train.columns)
                    Y_train = Y_train_r
                    
                    print("--- Se hizo resampleo para " + iterable +' en '+ Jcountry + " ---")
                    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                    pass
                    #Smoteen
                    #SMOTEENN(random_state=0)
                
                else:
                    pass


                #Training 
                # Model
                mod = model#**init_mod_kwargs)
                print("--- Se inicializó el modelo para " + iterable +' en '+ Jcountry + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))

                # Fit model
                mod.fit(X_train,Y_train)
                print("--- Se entrenó el modelo para " + iterable +' en '+ Jcountry + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                
                # Reporting
                # Maximize one metric
                maximizer = metric
                thresholds = []
                p = mod.predict_proba(X_test)[:,1]
                
                # Threshold space
                for thresh in np.arange(0.1, 0.801, 0.001):
                    res = maximizer(Y_test, (p > thresh).astype(int))#, beta=0.01)
                    thresholds.append([thresh, res])
                    
                thresholds.sort(key=lambda x: x[1], reverse=True)
                
                best_thresh =  np.round(thresholds[0][0], 4)
                best_metric =  np.round(thresholds[0][1], 4)
                
                print("--- Best score: " + str(best_thresh) +". Metric: " + str(best_metric) + " en "+Jcountry + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                
                #Save th for prediction
                Adt.save(Jcountry, best_thresh, path, model_name+'/'+iterable,'th_{}'.format(today))   
                print("--- Se guardó el treshold como el promedio de Y para " + iterable +' en '+ Jcountry + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                
                save_reporting(Jcountry, 
                               Y_test, 
                               X_test, 
                               mod, 
                               best_thresh, 
                               today, 
                               model_name, 
                               iterable, 
                               path=path, 
                               matrix_labels=[0,1])
                print("--- Se ha guardado el Reporting para " + iterable +' en '+ Jcountry + " ---")
                print("--- %s seconds ---\n" % round((time.time() - start_time),2))           

                #Save model for prediction
                Adt.save(Jcountry, mod, path, model_name+'/'+iterable,'mod_{}'.format(today))
                print("--- Se guardó el modelo para " + iterable +' en '+ Jcountry + " ---")
                print("--- %s seconds ---\n\n\n" % round((time.time() - start_time),2))
                
                del X_train, X_test, Y_train, Y_test
                gc.collect()
            else:
                iterables = np.delete(iterables,np.argwhere(iterables==iterable))
                print("--- No se hará modelo para " + iterable +' en '+ Jcountry + " por falta de información ---")
                print("--- %s seconds ---\n\n\n" % round((time.time() - start_time),2))
                
                
        Adt.save(Jcountry, iterables, path, model_name+'/', 'iterables_{}'.format(today))
        print("--- Se correrá un modelo para las iterablees " + iterables + " ---")
        print("--- %s seconds ---\n" % round((time.time() - start_time),2))
                

In [ ]:
init_mod_kwargs = { 'learning_rate':0.2
                                   ,'n_estimators':50
                                   ,'min_child_weight':1
                                   ,'gamma':0
                                   ,'subsample':0.8
                                   ,'colsample_bytree':0.8
                                   ,'objective':'binary:logistic'
                                   ,'scale_pos_weight':1
                                   ,'nthread':4 
                                   ,'seed':1144}
                
param_test1 = {'scale_pos_weight':range(50,110,20)}
                
gsearch1 = GridSearchCV(estimator = XGBClassifier(**init_mod_kwargs), 
                                    param_grid=param_test1, 
                                    scoring='roc_auc',
                                    n_jobs=4,
                                    iid=False, 
                                    cv=3)

In [ ]:
cv_params = {'learning_rate':[10,50,100], 
             'n_estimators':[800, 1000, 1700]}    # parameters to be tries in the grid search
fix_params = {'max_depth':4, 
              'min_samples_split':2, 
              'min_samples_leaf':1, 
              'subsample':1,
              'max_features':'sqrt'}   #other parameters, fixed for the moment 
csv = GridSearchCV(GradientBoostingClassifier(**fix_params), cv_params, scoring = 'f1', cv = 3)

In [ ]:
csv.fit(X_train, Y_train)

In [ ]:
csv.cv_results_, csv.best_params_, csv.best_score_

In [ ]:
csv.best_params_

In [ ]:
X_train

In [ ]:
#qh
query_high = '''

SELECT T0.*,
       LEFT(RIGHT(try_to_number(APPLICATION_USER_ID),UNIFORM(2,3,HOUR(CURRENT_TIMESTAMP))),2) + LEFT(RIGHT(UNIFORM(1.00::FLOAT,100::FLOAT,HOUR(CURRENT_TIMESTAMP)),5),2) AS RANDOM
FROM [Jcountry]_WRITABLE.[Jtable] T0
--WHERE [Jiterable_name] = 6--[Jiterable]
ORDER BY RANDOM
limit 200000


'''

In [ ]:
q = query_high.replace('[Jtable]','high_value_user_revenue_data_microzone_exp')
q = q.replace('[Jcountry]','BR')
ALC_grid = Jsc.pandas_df_from_snowflake_query(con, q)

In [ ]:
q = query_high.replace('[Jtable]','high_value_user_revenue_data_microzone_exp')
q = q.replace('[Jcountry]','BR')
ALC = Jsc.pandas_df_from_snowflake_query(con, q)

In [ ]:
ALC_model = ALC_grid.copy()

In [ ]:
del ALC_model
gc.collect()

In [ ]:
#exclude1
exclude1 = ['QTY_SELECT_LOGIN_TOTAL_ZZZ',
 'QTY_SELECT_LOGIN_ZZZ',
 'ORDER_PLACED_CONFIRMED_ERROR_FOUND_ZZZ',
 'QTY_ORDER_PC_ERROR_TOTAL_ZZZ',
 'QTY_COUPON_ERROR_TOTAL_ZZZ',
 'QTY_COUPON_ERROR_ZZZ',
 'COMPLETE_SIGN_ERROR_FOUND_ZZZ',
 'QTY_COMPLETE_SIGN_ERROR_TOTAL_ZZZ',
 'COMPLETE_REGISTRATION_FOUND_ZZZ',
 'SESSIONS_FOUND_EXX',
'QTY_COMPLETE_LOGIN_ERROR_ZZZ',
 'COMPLETE_LOGIN_FOUND_ZZZ',
 'QTY_COMPLETE_LOGIN_ZZZ',
 'ADDRESS_CHECK_ERROR_FOUND_ZZZ',
 'QTY_ADDRESS_CHECK_ERROR_ZZZ',
 'ADDRESS_CHANGED_FOUND_ZZZ',
 'QTY_ADDRESS_CHANGED_ZZZ',
 'ADD_ADDRESS_ERROR_FOUND_ZZZ',
 'QTY_VIEW_HOME_ERROR_ZZZ',
 'COUPON_ERROR_FOUND_ZZZ',
'QTY_COMPLETE_REGISTRATION_ZZZ',
 'COMPLETE_REGISTER_ERROR_FOUND_ZZZ',
 'QTY_COMP_REGI_ERROR_TOTAL_ZZZ',
 'QTY_COMPLETE_REGISTER_ERROR_ZZZ',
 'COMPLETE_LOGIN_ERROR_FOUND_ZZZ',
 'QTY_ADD_ADDRESS_ERROR_TOTAL_ZZZ',
 'MODE_OS_KKK',
 'QTY_ADD_ADDRESS_ERROR_ZZZ',
 'VERIFICATION_CODE_ERROR_FOUND_ZZZ',
 'QTY_VERIFICATION_CODE_SUCCESS_ZZZ',
'CARD_BRAND_AGG_KKK', 'ACTUAL_TIP_SHARE_ZZZ',
       'DIF_ORGANIC_SHARE_ZZZ', 'MODE_PAYMENT_METHOD_KKK', 'TRIES_ZZZ',
       'DIF_FIRST_DEVICE_YEAR_ZZZ', 'REGISTER_TRIES_ZZZ',
       'NUMBER_OF_CC_ZZZ', 'LOG_AVG_DURATION_MS_AVG_TOTAL_ZZZ',
       'WEEK_REGISTER_CC_EXX', 'VIEW_HOME_ERROR_FOUND_ZZZ',
       'QTY_VIEW_HOME_ERROR_TOTAL_ZZZ',
       'VERIFICATION_CODE_SUCCESS_FOUND_ZZZ',
       'LOG_VERIFICATION_CODE_RATIO_ZZZ', 'VERIFICATION_CODE_RATIO_ZZZ',
       'QTY_VERIFICATION_CODE_ERROR_ZZZ', 'QTY_COMPLETE_SIGN_ERROR_ZZZ',
       'RELATIVE_DIFF_MS_DURATION_ZZZ', 'MAX_TIP_SHARE_ZZZ',
       'SAME_COUNTRY_EXX']

In [ ]:
new_out = [ 'AVG_REV_ORD_MIC_ZZZ'
            ,'MED_REV_ORD_MIC_ZZZ'
            ,'TOT_REV_MIC_ZZZ'
            ,'TOT_ORD_MIC_ZZZ'
            ,'TOT_USERS_MIC_ZZZ'
            ,'PERC_MICRO_REV_MIC_EXX'
            ,'PERC_MICRO_ORD_MIC_EXX'
            ,'HV_MICROZONE_MIC_EXX'
            ,'AVG_ORD_USER_MIC_ZZZ' #Micro
            ,'AVG_REV_USER_MIC_ZZZ' #Micro
            ,'STD_REV_ORD_MIC_ZZZ'] #Micro

In [ ]:
exclude1 = exclude1 + new_out

In [ ]:
experimental = [ 'PERC_FIRST_REVE'
                ,'ACTUAL_FIRST_REVENUE']

numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]#+[experimental[1]]
cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra] #+[experimental[0]]



In [ ]:
numeric_feat = [num for num in numeric_feat if num not in exclude1]
cate_feat_encode = [cat for cat in cate_feat_encode if cat not in exclude1]
extra_feat = [extra for extra in extra_feat if extra not in exclude1]

In [ ]:
ALC_treat = ALC_model[numeric_feat + cate_feat_encode + extra_feat]

            
#Treat numeric null values
ALC_treat = Adt.Nan_Numeric(ALC_treat,numeric_feat)

ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)
ALC_treat = Adt.Nan_Cate(ALC_treat,extra_feat,replacement=-1)

In [ ]:
ALC_treat.shape

In [ ]:
sc = StandardScaler()
sc.fit(ALC_treat[numeric_feat])        
ALC_treat[numeric_feat] = pd.DataFrame(sc.transform(ALC_treat[numeric_feat]), columns=numeric_feat, index = ALC_treat.index)
        
#Define X and Target
y = ALC_treat['HIGH_VALUE_USER_EXX']
X = ALC_treat.drop(['HIGH_VALUE_USER_EXX'],axis=1)
            
#Split training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 27, stratify=y)

In [ ]:
cate_low = CatBoostEncoder(cols = cate_feat_encode, drop_invariant=False)
cate_low.fit(X_train, Y_train, cols = cate_feat_encode)
X_train = cate_low.transform(X_train, Y_train)
                    
#cate_high = LeaveOneOutEncoder(cols = high_cardinality, drop_invariant=False)
#cate_high.fit(X_train, Y_train, cols = high_cardinality)
#X_train = cate_high.transform(X_train)
            
            
#X_test = cate_high.transform(X_test)
X_test = cate_low.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
#PCA
#from sklearn.decomposition import PCA
# Make an instance of the Model
#pca = PCA(.95)
#pca.fit(X_train)
#X_train = pca.transform(X_train)
#X_test = pca.transform(X_test)

In [ ]:
#oversampling
#sampler = ADASYN(random_state=0)#ADASYN
                    
#X_train_r, Y_train_r = sampler.fit_resample(X_train, Y_train)
                    
#X_train = pd.DataFrame(X_train_r, columns=X_train.columns)
#Y_train = Y_train_r



X_train.shape

In [ ]:
mod = GradientBoostingClassifier()

In [ ]:
mod.fit(X_train,Y_train)

In [ ]:
sum(Y_train)/len(Y_train)

In [ ]:
roc_auc_score(Y_test, mod.predict_proba(X_test)[:,1])

In [ ]:
mod.predict_proba(X_test)[:,1]

In [ ]:
maximizer = fbeta_score
thresholds = []
p = mod.predict_proba(X_test)[:,1]

# Threshold space
for thresh in np.arange(0.1, 0.801, 0.001):
    res = maximizer(Y_test, (p > thresh).astype(int), beta=0.8)
    thresholds.append([thresh, res])
                    
thresholds.sort(key=lambda x: x[1], reverse=True)
                
best_thresh =  np.round(thresholds[0][0], 4)
best_metric =  np.round(thresholds[0][1], 4)

In [ ]:
print(best_thresh, best_metric)

In [ ]:
th = best_thresh
p=mod.predict_proba(X_test)[:,1]
p[p<th]=0
p[p>=th]=1
print(classification_report(Y_test,p))

In [ ]:
#All
arg_s = (-mod.feature_importances_).argsort()
data1 = pd.DataFrame(zip(X_test.columns[arg_s] , mod.feature_importances_[arg_s]), columns = ['COLUMNA','IMPORTANCIA'])
data1.IMPORTANCIA = round(data1.IMPORTANCIA,3)
#data.COLUMNA = data['COLUMNA'].map(lambda x: x.replace('KKK','').replace('ZZZ','').replace('EXX','')) 
#data.COLUMNA = data['COLUMNA'].map(lambda x: '\n '.join(x[i:i + 15] for i in range(0, len(x), 15))) 

In [ ]:
data1.tail(20).COLUMNA.values

In [ ]:
init = {'n_estimators':800}
export_train(query_promo=query_high, 
             model_name='high_value_users_new', 
             table_name_train='high_value_user_revenue_data',
             iterable_name='TIME_WINDOW',
             target_name='HIGH_VALUE_USER_EXX',
             resample = False,
             resampler = SMOTEENN(random_state=0),
             path='../data_1/',
             model=GradientBoostingClassifier(**init),#XGBClassifier(), 
             countries=['BR', 'CO', 'MX', 'CL', 'AR', 'PE', 'UY'])

In [ ]:
#Bactest tryout
query_back = '''
SELECT *
FROM [Jcountry]_writable.high_value_user_revenue_data_backtest

'''

In [ ]:
countries = {'BR': 5, 'MX': 4, 'CO': 5, 'CL': 4, 'AR': 5}#{'BR': 5, 'MX': 4, 'CO': 5, 'PE': 5, 'CL': 4, 'AR': 5, 'UY': 5}
start_time = time.time()
today = time.strftime('%Y-%m-%d', time.localtime(start_time))
print("--- Se hará el entrenamiento el día " + today + " ---")
print("--- %s seconds ---\n" % round((time.time() - start_time),2))
for country in zip(countries.keys(),countries.values()):
    
    query1 = query_back.replace('[Jcountry]',country[0])
    
    ALC_model = Jsc.pandas_df_from_snowflake_query(con, query1)
    print('--- Cargados los datos crudos para ' + country[0] + ' ---')
    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
    
    numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]
    cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
    extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra]
            
    numeric_feat = [num for num in numeric_feat if num not in exclude1]
    cate_feat_encode = [cat for cat in cate_feat_encode if cat not in exclude1]
    extra_feat = [extra for extra in extra_feat if extra not in exclude1]

    ALC_treat = ALC_model[numeric_feat + cate_feat_encode + extra_feat]

    ALC_treat[numeric_feat] = ALC_treat[numeric_feat].astype(float)

    #Treat numeric null values
    ALC_treat = Adt.Nan_Numeric(ALC_treat,numeric_feat)
    ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)
    ALC_treat = Adt.Nan_Cate(ALC_treat,extra_feat,replacement=-1)
        
    X = ALC_treat.drop("HIGH_VALUE_USER_EXX",axis=1)
        
    path_cate_high = '../data_1/{}/high_value_users_new/{}/cate_high_2020-04-23'.format(country[0],str(country[1]))
    path_cate_low = '../data_1/{}/high_value_users_new/{}/cate_low_2020-04-23'.format(country[0],str(country[1]))
    path_sc = '../data_1/{}/high_value_users_new/{}/scaler_2020-04-23'.format(country[0],str(country[1]))
    path_mod = '../data_1/{}/high_value_users_new/{}/mod_2020-04-23'.format(country[0],str(country[1]))
    path_th = '../data_1/{}/high_value_users_new/{}/th_2020-04-23'.format(country[0],str(country[1]))
        
        
    cate_high = Adt.read(path_cate_high)
    cate_low = Adt.read(path_cate_low)
    sc = Adt.read(path_sc)
    mod = Adt.read(path_mod)
    th = Adt.read(path_th)
        
    print('--- Cargados los pkl'+ ' ---')
    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
    X[numeric_feat] = pd.DataFrame(sc.transform(X[numeric_feat]), columns=numeric_feat, index = X.index)
    X = cate_high.transform(X)
    X = cate_low.transform(X)
        
    y = ALC_treat["HIGH_VALUE_USER_EXX"]
        
    print('--- Transformaciones listas'+ ' ---')
    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
        
    save_reporting(country[0], 
                   y, 
                   X , 
                   mod, 
                   th, 
                   today='back_test', 
                   model_name='high_value_users_new', 
                   iterable=str(country[1]), 
                   path='../data_1/', 
                   matrix_labels=[0,1])
        
        
    print('--- Guardado el reporting'+ ' ---')
    print("--- %s seconds ---\n" % round((time.time() - start_time),2))
        
    ALC_model['PRED_' + str(country[1])] = mod.predict_proba(X)[:, 1]

    ALC_model['TRH_' + str(country[1])] = [th] * ALC_model.shape[0]
        
    ALC_model['REAL_HVU_' + str(country[1])] = ALC_model["HIGH_VALUE_USER_EXX"]
        
        
    export = [i for i in ALC_model.columns if 'PRED_' in i] + \
             [i for i in ALC_model.columns if 'TRH_' in i] + \
             [i for i in ALC_model.columns if 'REAL_HVU_' in i] + \
             [i for i in ALC_model.columns if ('PREV' in i) & ('LAUNCH' not in i)] + \
             [i for i in ALC_model.columns if 'PORD' in i] + \
             [i for i in ALC_model.columns if 'PERC' in i] + \
             ['APPLICATION_USER_ID'] + ['FIRST_ACTUAL_ORDER', 'TIME_WINDOW']
    
    
    resp = Jsc.write_snowflake_table(data=ALC_model[export],
                                        table_name='high_value_user_pred_backtest',
                                        Jcountry = country[0],
                                        user=user,
                                        password = password,
                                        if_exists_then_wat='replace')
    
    del ALC_model
    gc.collect()
                                                
    print('--- Exportado a high_value_user_pred_backtest para ' + country[0]+ ' ---')
    print("--- %s seconds ---\n\n" % round((time.time() - start_time),2))


In [ ]:
ALC_model[export].columns

In [ ]:
ALC_model = Jsc.pandas_df_from_snowflake_query(con, query)

In [ ]:
ALC_model.memory_usage()

In [ ]:
ALC_model.APPLICATION_USER_ID.values[0]

In [ ]:
numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]
cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra]

In [ ]:
ALC_treat = ALC_model[numeric_feat + cate_feat_encode + extra_feat]

In [ ]:
ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)

In [ ]:
y = ALC_treat['HIGH_VALUE_USER_EXX']
X = ALC_treat.drop(['HIGH_VALUE_USER_EXX'],axis=1)

In [ ]:
X[cate_feat_encode].head()

In [ ]:
#Category encoding
high_cardinality = []
low_cardinality = []
            
for cat_column in cate_feat_encode:
    cates = len(ALC_treat[str(cat_column)].drop_duplicates().values)
    if (cates/len(cate_feat_encode)) > 0.25:
        high_cardinality.append(cat_column)
    else:
        low_cardinality.append(cat_column)
                    
cate_high = LeaveOneOutEncoder(cols=high_cardinality+low_cardinality, drop_invariant=True)
cate_high.fit(X,y,cols=high_cardinality+low_cardinality)
X = cate_high.transform(X,y)
            
#cate_low = OneHotEncoder(use_cat_names=True)
#cate_low.fit(X,y,cols=low_cadinality)

In [ ]:
a = ['a'] + []

In [ ]:
a

In [ ]:
numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]
cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra]

In [ ]:
ALC_treat = ALC_model[numeric_feat + cate_feat_encode + extra_feat]

            #Treat numeric null values
ALC_treat = Adt.Nan_Numeric(ALC_treat,numeric_feat)

ALC_treat = Adt.Nan_Cate(ALC_treat,cate_feat_encode)

sc = StandardScaler()
sc.fit(ALC_treat[numeric_feat])        
ALC_treat[numeric_feat] = pd.DataFrame(sc.transform(ALC_treat[numeric_feat]), columns=numeric_feat, index = ALC_treat.index)


In [ ]:
y = ALC_treat['HIGH_VALUE_USER_EXX']
X = ALC_treat.drop(['HIGH_VALUE_USER_EXX'],axis=1)
            
            
            #Split training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 27, stratify=y)
            

            #Category encoding
high_cardinality = []
low_cardinality = []
            
for cat_column in cate_feat_encode:
    cates = len(X_train[str(cat_column)].drop_duplicates().values)
    if (cates/len(cate_feat_encode)) > 20:
        high_cardinality.append(cat_column)
        print(cat_column,cates/len(cate_feat_encode))
    else:
        low_cardinality.append(cat_column)
        print(cat_column,cates/len(cate_feat_encode))
                    
#cate_high = LeaveOneOutEncoder(cols = high_cardinality, drop_invariant=True)
#cate_high.fit(X_train, Y_train, cols = high_cardinality)

#cate_low = LeaveOneOutEncoder(cols = low_cardinality, drop_invariant=False)
#cate_low.fit(X_train, Y_train, cols = low_cardinality)

In [ ]:
high_cardinality

In [ ]:
X_train=cate_low.transform(X_train)

In [ ]:
X_train[low_cardinality]

In [ ]:
len(cate_feat_encode)

In [ ]:
len(ALC_model.APPLICATION_USER_ID.drop_duplicates().values)

In [ ]:
ALC_model.shape

In [ ]:
y = mod.predict_proba(X_test)[:,1]

y[y<th_org]=0
y[y>=th_org]=1

In [ ]:
X_train

In [ ]:
sns.countplot(y)

In [ ]:
sns.countplot(Y_test)

In [ ]:
numeric_feat = [num for num in ALC_model.columns if 'ZZZ' in num]
cate_feat_encode = [cat for cat in ALC_model.columns if 'KKK' in cat]
extra_feat = [extra for extra in ALC_model.columns if 'EXX' in extra]

In [ ]:
tl = ['AGE_RANGE_KKK','HIGH_VALUE_USER_EXX']
why = ALC_model.copy()#[tl]

In [ ]:
X_transf = cate_high.transform(why[numeric_feat+cate_feat_encode+extra_feat].drop(['HIGH_VALUE_USER_EXX'],axis=1))

In [ ]:
X_transf['TARGET'] = why['HIGH_VALUE_USER_EXX']

In [ ]:
sns.countplot(X_transf['HIGH_VALUE_USER_EXX'])

In [ ]:
ALC_model['LAST_VERTICAL_KKK'][ALC_model.LAST_VERTICAL_KKK=='GAMERS']

In [ ]:
X_transf[X_transf.index==151388].LAST_VERTICAL_KKK#.drop_duplicates()

In [ ]:
sns.scatterplot(x='TARGET',y='LAST_VERTICAL_KKK',data=X_transf)

In [ ]:
y_pre = mod.predict_proba(X_test)[:,1]

In [ ]:
y_pre

In [ ]:
roc_auc_score(Y_test,y_pre)

In [ ]:
#query_automation
query_model = '''

SELECT T0.*,
       DAYNAME(CREATED_AT) AS DAY_WEEK,
       CASE WHEN DAYNAME(CREATED_AT) IN ('Sat','Sun') THEN 1 ELSE 0 END AS WEEKEND,
       LEFT(RIGHT(try_to_number(APPLICATION_USER_ID),UNIFORM(2,3,HOUR(CURRENT_TIMESTAMP))),2) + LEFT(RIGHT(UNIFORM(1.00::FLOAT,100::FLOAT,HOUR(CURRENT_TIMESTAMP)),5),2) AS RANDOM
FROM [Jcountry]_WRITABLE.VERTICAL_CROSS_DATA T0
WHERE RECENCY IS NOT NULL
AND LENGTH(APPLICATION_USER_ID) >= 3
AND LEFT(RIGHT(APPLICATION_USER_ID,2),1) NOT IN (1,5,9)
ORDER BY RANDOM
limit 1000000

'''

query_model_momentum = '''

SELECT T0.*,
       DAYNAME(CREATED_AT) AS DAY_WEEK,
       CASE WHEN DAYNAME(CREATED_AT) IN ('Sat','Sun') THEN 1 ELSE 0 END AS WEEKEND,
       LEFT(RIGHT(try_to_number(APPLICATION_USER_ID),UNIFORM(2,3,HOUR(CURRENT_TIMESTAMP))),2) + LEFT(RIGHT(UNIFORM(1.00::FLOAT,100::FLOAT,HOUR(CURRENT_TIMESTAMP)),5),2) AS RANDOM
FROM [Jcountry]_WRITABLE.VERTICAL_CROSS_DATA T0
WHERE RECENCY IS NOT NULL
AND PEDIDOS <= 5
AND LENGTH(APPLICATION_USER_ID) >= 3
AND LEFT(RIGHT(APPLICATION_USER_ID,2),1) NOT IN (1,5,9)
ORDER BY RANDOM
limit 1000000

'''

In [ ]:
#momentum
export_train(query_model_momentum,
             'cross_mod_momentum', 
             ['PEDIDOS','VERTICALS','GMV_CUMULATIVE','GMV_MEDIAN','GMV_MIN','GMV_RANGE','TICKET_CUMULATIVE','TICKET_MEDIAN','TICKET_MIN','TICKET_RANGE', 'TIP_CUMULATIVE' , 'TIP_MEDIAN' ,'TIP_MIN','TIP_RANGE','AVG_POND_NEW','MEDIAN_DIST','ORGANIC_RATIO','TIP_RATIO','RECENCY'], 
             ['DAY_WEEK','MODE_VERTICAL','FIRST_VERTICAL','LAST_VERTICAL','MODE_PAYMENT','FIRST_PAYMENT','LAST_PAYMENT','MODE_OS','FIRST_OS','LAST_OS'], 
             ['WEEKEND','CROSS_SELL'],
             countries=['CO','BR','MX','PE','CL','AR']
            )

In [ ]:
#conso
export_train(query_model,
             'cross_mod', 
             ['PEDIDOS','VERTICALS','GMV_CUMULATIVE','GMV_MEDIAN','GMV_MIN','GMV_RANGE','TICKET_CUMULATIVE','TICKET_MEDIAN','TICKET_MIN','TICKET_RANGE', 'TIP_CUMULATIVE' , 'TIP_MEDIAN' ,'TIP_MIN','TIP_RANGE','AVG_POND_NEW','MEDIAN_DIST','ORGANIC_RATIO','TIP_RATIO','RECENCY'], 
             ['DAY_WEEK','MODE_VERTICAL','FIRST_VERTICAL','LAST_VERTICAL','MODE_PAYMENT','FIRST_PAYMENT','LAST_PAYMENT','MODE_OS','FIRST_OS','LAST_OS'], 
             ['WEEKEND','CROSS_SELL'],
             countries=['UY']
            )

**Beyond this point is how things get done before automation**

**********************************************************************************

Yo do not need to go there, but do it if you want


*******************************************************************************

.
*******************************************************************************


.
**********************************************************************************


:)
**********************************************************************************

.
**********************************************************************************


.
**********************************************************************************



In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pre)
roc_auc = auc(fpr, tpr)
plot(fpr, tpr, lw=1, alpha=0.3,label='(AUC = %0.2f)'%roc_auc)
plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
set_xlim([-0.05, 1.05])
set_ylim([-0.05, 1.05])
set_xlabel('False Positive Rate')
set_ylabel('True Positive Rate')
set_title('ROC curve ')
legend(loc='best')

In [ ]:
a

In [ ]:
a = np.array(['Holi','Aaaa'])
index = np.argwhere(a=='Holi')
y = np.delete(a, index)

In [ ]:
y

# Classification Model

In [ ]:
query = '''

SELECT *
FROM BR_WRITABLE.HIGH_VALUE_DATA


'''

In [ ]:
ALC_model = Jsc.pandas_df_from_snowflake_query(con, query)

In [ ]:
#ALC_model['CREATED_AT'] = pd.to_datetime(ALC_model.CREATED_AT)
ALC_model['DAY_WEEK'] = [i.weekday() for i in ALC_model.CREATED_AT.values]
ALC_model['WEEKEND'] = [0]*len(ALC_model['DAY_WEEK'])
ALC_model['WEEKEND'][(ALC_model.DAY_WEEK == 5) & (ALC_model.DAY_WEEK == 6)] = 1

In [ ]:
#ALC_model['CREATED_AT'] = pd.to_datetime(ALC_model.CREATED_AT)
ALC_model['DAY_WEEK'] = [i.weekday() for i in ALC_model.CREATED_AT.values]
ALC_model['WEEKEND'] = [0]*len(ALC_model['DAY_WEEK'])
ALC_model['WEEKEND'][(ALC_model.DAY_WEEK == 5) & (ALC_model.DAY_WEEK == 6)] = 1

In [ ]:
ALC_model.columns

## Data treatment

In [ ]:
numeric_feat = ['PEDIDOS','VERTICALS','GMV_CUMULATIVE','GMV_MEDIAN','GMV_MIN','GMV_RANGE','TICKET_CUMULATIVE','TICKET_MEDIAN','TICKET_MIN','TICKET_RANGE', 'TIP_CUMULATIVE' , 'TIP_MEDIAN' ,'TIP_MIN','TIP_RANGE','AVG_POND_NEW','MEDIAN_DIST','ORGANIC_RATIO','TIP_RATIO','RECENCY']                                                
cate_feat_encode = ['DAY_WEEK','MODE_VERTICAL','FIRST_VERTICAL','LAST_VERTICAL','MODE_PAYMENT','FIRST_PAYMENT','LAST_PAYMENT','MODE_OS','FIRST_OS','LAST_OS']
extra_feat = ['WEEKEND','CROSS_SELL']

ALC_treat = ALC_model[ALC_model.VERTICAL_SUB_GROUP=='EXP'][numeric_feat + cate_feat_encode + extra_feat]
ALC_treat.head()

In [ ]:
#Treat numeric null values
ALC_treat[numeric_feat].fillna(np.mean(ALC_treat[numeric_feat]),inplace=True)

In [ ]:
#Data pro
#Numeric scaling
#sc = StandardScaler()

#sc.fit(ALC_treat[numeric_feat])
ALC_treat[numeric_feat] = pd.DataFrame(sc.transform(ALC_treat[numeric_feat]), columns=numeric_feat, index = ALC_treat.index)

#Define X and Target
y = ALC_treat['CROSS_SELL']
X = ALC_treat.drop(['CROSS_SELL'],axis=1)



#Category encoding
#cate = OneHotEncoder()
#cate.fit(X,y,cols=cate_feat_encode)
X = cate.transform(X,y)


#Save scaler for prediction
#Adt.save(Jcountry, sc, '../Data/' , 'cross_model', 'scaler')
#Adt.save(Jcountry, cate, '../Data/','cross_model', 'cate')

#Split training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 27, stratify=y)

''' 
#Resampling
#Smote
smote = SMOTE(random_state=0)
X_tra_re, Y_tra_re = smote.fit_resample(X_train, Y_train)

#Smoteen
smote_enn = SMOTEENN(random_state=0)
X_tra_re1, Y_tra_re1 = smote_enn.fit_resample(X_train, Y_train)
'''
#Tresholds
#th_org = sum(Y_train)/len(Y_train)

#Save th for prediction
#Adt.save(Jcountry, th_org, '../Data/' , 'cross_model','th')


In [ ]:
Adt.save(Jcountry, th_org, '../Data/' , 'cross_model','th')
Adt.save(Jcountry, sc, '../Data/' , 'cross_model', 'scaler')
Adt.save(Jcountry, cate, '../Data/','cross_model', 'cate')

In [ ]:
#Model selection
#Create an array of models
models = []
models.append(("GB",GradientBoostingClassifier()))
models.append(("RF",RandomForestClassifier()))
#models.append(("SVC",svc()))
models.append(("XGB",XGBClassifier()))

#Measure the some metric 
#To check what metrics are available use: sorted(SCORERS.keys())
for name, model in models:
    kfold = KFold(n_splits=3, random_state=22)
    cv_result = cross_val_score(model,X_train,Y_train, cv = kfold, scoring = "roc_auc")
    print(name, cv_result)

In [ ]:
#Training 

init_mod_kwargs = { 'n_estimators':30
                   ,'min_samples_split':50
                   ,'min_samples_leaf':8
                   ,'max_depth':20
                   ,'criterion':'gini'
                   ,'max_features':'auto'
                   ,'bootstrap':True
                   ,'n_jobs':-1
                   ,'random_state':2305
                   ,'verbose':2 }
# Model
mod = XGBClassifier()#**init_mod_kwargs)

# Fit model
mod.fit(X_train,Y_train)

#Save model for prediction
#Adt.save(Jcountry, mod, '../../Data/' , 'cross_model','mod')

## Results report

In [ ]:
mod = Adt.read('../../Data/UY/cross_mod/verticals')
#th_org = Adt.read('../../Data/UY/cross_mod/EXP/th')
#cate = Adt.read('../../Data/UY/cross_mod/EXP/cate')
#sc = Adt.read('../../Data/UY/cross_mod/EXP/scaler')

In [ ]:
mod

In [ ]:
np.delete(mod,np.argwhere(mod=='MAR'))

In [ ]:
# Ordena las columnas de menor a mayor importancia
arg_s = (-mod.feature_importances_).argsort()
best_cols = pd.DataFrame(zip(X_train.columns[arg_s] ,mod.feature_importances_[arg_s]),columns = ['COLUMNA','IMPORTANCIA'])

In [ ]:
import pandas as pd
a = pd.DataFrame([[1,2,3],[1,2,3]],columns=['A','B','C'])

In [ ]:
a.shape[0]

In [ ]:
p = mod.predict_proba(X_test)[:,1]
p[p<th_org] = 0
p[p>=th_org] = 1

In [ ]:
Adt.save('UY', a, '../../Data/' , 'cross_mod'+'/'+'EXP','class')

In [ ]:
import ml_insights as mli
#xray = mli.ModelXRay(mod, X_test)

In [ ]:
X_train.columns

In [ ]:
p = np.random.choice([1,0],1000)
Y_test = np.random.choice([1,0],1000)

In [ ]:
a = classification_report(p,Y_test)


In [ ]:
report = classification_report(p, Y_test, output_dict=True)

In [ ]:
df = pd.DataFrame(report).transpose()

In [ ]:
df.to_csv(r'C:\Users\Usuario\conso_vertical_cross\Data\UY\cross_mod\LIC\Reporting\df.csv')

In [ ]:
confusion_matrix(Y_test, p)

In [ ]:
Adt.save('UY', 'This is just to save coding time :)', '../../Data/' , 'cross_mod'+'/'+'EXP','class')

In [ ]:
a = plotLiftChart(Y_test, mod.predict_proba(X_test)[:,1],'../../Data/'+'UY'+'/'+'cross_mod'+'/'+'EXP'+'/'+'lift')

In [ ]:
plot_roc_curve(Y_test,mod.predict_proba(X_test)[:,1])

In [ ]:
arg_s = (-mod.feature_importances_).argsort()
best_cols = list([X_test.columns[arg_s[:50]]][0])
bad_cols = list([X_test.columns[arg_s[-50:]]][0])


In [ ]:
mod.feature_importances_

In [ ]:
best_cols


In [ ]:
# Genera el reporte de performance de acuerdo a los puntos definidos anteriormente
# Input: X: las variables predictivas, y: la variable target, model: el modelo entrenado
def performance_report(X,y,model,folder=None):
    # primero se obtiene la predicción
    # Se obtiene la probabilidad en lugar de la clasificación final porque tiene más información y es más versátil
    y_model = model.predict_proba(X)[:,1] # La probabilidad de la clase '1' (casualmente el índice 1 en la lista [0,1])
    ##############################################################
    ################ MAXIMIZACION DEL F1 SCORE ###################
    ##############################################################
    thresholds = []
    # Recorre el espacio de thresholds (para en 0.4) dado el desbalanceo del target (~20%)
    for thresh in np.arange(0.1, 0.401, 0.001):
        # Calcula el f1 en el threshold actual
        res = f1_score(y, (y_model > thresh).astype(int))
        # añade los resultados a la lista
        thresholds.append([thresh, res])
    # Ordena los resultados de mayor F1 a menor F1
    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh =  np.round(thresholds[0][0], 3)
    print('-----------------------------------------------------------------')
    print('El trheshold que maximiza el F1 score es: ', best_thresh)
    print('-----------------------------------------------------------------')
    # La prediccion en términos de clases
    y_pred_class = (y_model > thresh).astype(int)
    ##############################################################
    ################ AREA UNDER THE (ROC) CURVE ##################
    ##############################################################
    # Tasa de falsos positivos (false positive rate) y tasa de verdaderos positivos (true positive rate)
    fpr, tpr, thresholds = roc_curve(y, y_model)
    auc_val = auc(fpr, tpr)
    print('-----------------------------------------------------------------')
    print('El valor de AUC es: ', auc_val)
    print('-----------------------------------------------------------------')
    ##############################################################
    ################### BALANCED_ACCURACY ########################
    ############################################################## 
    bas = balanced_accuracy_score(y, y_pred_class)
    print('-----------------------------------------------------------------')
    print('El valor del Balanced Accuracy es: ', bas)
    print('-----------------------------------------------------------------')
    ##############################################################
    ############## REPORTE DE CLASIFICACION ######################
    ############################################################## 
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')
    classif_rep = classification_report(y, y_pred_class)
    print(classif_rep)
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')
    # CREA LA CARPETA
    if not os.path.exists(folder):
        os.makedirs(folder)
    cpickle.dump(classif_rep, open(folder+"classification_report_"+dt.today().strftime('%Y-%m-%d')+".pickle", 'wb'))  
    # Crea la figura
    fig = plt.figure(figsize=(12,8))
    #plt.gcf().add_subplot()
    # Define los ejes
    ax1 = plt.subplot(221)
    ax3 = plt.subplot(212)
    ax2 = plt.subplot(222)
    ##############################################################
    ################### GRAFICA LA ROC ###########################
    ############################################################## 
    # Compute ROC curve and area the curve
    ax1.plot(fpr, tpr, lw=1, alpha=0.3,label='(AUC = %0.3f)'%auc_val)
    ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    ax1.set_xlim([-0.05, 1.05]) #plt.xlim
    ax1.set_ylim([-0.05, 1.05]) 
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC curve ')
    ax1.legend(loc='best')
    ##############################################################
    # MATRIZ DE CONFUSION ( con el threshold que maximiza el f1) #
    ############################################################## 
    labels = [1, 0]
    cm = confusion_matrix(y, y_pred_class, labels)
    print(cm)
    cax = ax2.matshow(np.log(cm))
    fig.colorbar(cax)
    ax2.set_xticklabels([''] + labels)
    ax2.set_yticklabels([''] + labels)
    ax2.set_xlabel('Actual Class')
    ax2.set_ylabel('Predicted Class')
    ##############################################################
    ################ GRAFICA EL LIFT CHART #######################
    ############################################################## 
    df_dict = {'actual': list (y), 'pred': list(y_model)}
    df = pd.DataFrame(df_dict)
    pred_ranks = pd.qcut(df['pred'].rank(method='first'), 10, labels=False)
    actual_ranks = pd.qcut(df['actual'].rank(method='first'), 10, labels=False)
    pred_percentiles = df.groupby(pred_ranks).mean()
    actual_percentiles = df.groupby(actual_ranks).mean()
    ax3.set_title('Lift Chart')
    ax3.plot(np.arange(.1, 1.1, .1), np.array(pred_percentiles['pred']),
             color='darkorange', lw=2, label='Prediction')
    ax3.plot(np.arange(.1, 1.1, .1), np.array(pred_percentiles['actual']),
             color='navy', lw=2, linestyle='--', label='Actual')
    ax3.set_ylabel('Target Average')
    ax3.set_xlabel('Population Percentile')
    ax3.set_xlim([0.0, 1.1])
    ax3.set_ylim([0,0.05+max([max(np.array(pred_percentiles['pred'])),max(np.array(pred_percentiles['actual']))])])
    ax3.legend(loc="best")
    fig.tight_layout()
    #fig.savefig(folder+"roc_lift_confussion_report_"+dt.today().strftime('%Y-%m-%d')+".pdf")

In [ ]:
from xgboost import plot_tree
fig, ax = plt.subplots(figsize=(30, 30))
plot_tree(mod, num_trees=0, ax=ax)
plt.show()